In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen as uReq
from urllib.request import Request, urlopen
from unidecode import unidecode

import os
import re
import pickle
import json

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)

In [4]:
def get_all_ufc_cards():
    cards_links = []
    for x in range(5,0,-1):
        driver.get(f"https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/{x}")
        site =  BeautifulSoup(driver.page_source,'html.parser')
        for line in site.find('div',{'id':'recent_tab'}).find('tbody').find_all('tr')[::-1]:
            if line.find('a'):
                cards_links.append("https://www.sherdog.com"+line.a['href'])
    return cards_links

In [5]:
cards = get_all_ufc_cards()

In [6]:
fighters_links = {}

In [7]:
def get_all_ufc_fighters(ufc_card_links):
    for cnumber,card in enumerate(ufc_card_links):
        print(cnumber,end=' ')
        driver.implicitly_wait(1)
        driver.get(card)
        site = BeautifulSoup(driver.page_source,'html.parser')
        if not site.find('div',class_='module fight_card'): continue
        for main_card in site.find('div',class_='module fight_card').find_all('a'):
            if main_card.find('span'):
                if main_card.find('span').text not in fighters_links:
                    fighters_links[main_card.find('span').text] = main_card['href']
        
        for fight in site.find('div',class_='content table').find_all('tr',{'itemprop':'subEvent'}):
            for fighter in fight.find_all('td',{'itemprop':'performer'}):
                if fighter.a.text not in fighters_links:
                    fighters_links[fighter.a.text] = fighter.a['href']

In [8]:
get_all_ufc_fighters(cards)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [9]:
def sherdog_date_decoder(string):
    months = {'Jan': '01','Feb': '02','Mar': '03',
      'Apr': '04','May': '05','Jun': '06',
      'Jul': '07','Aug': '08','Sep': '09',
      'Oct': '10','Nov': '11','Dec': '12'}
    new_date = string.split('/')
    return new_date[2].strip()+'-'+months[new_date[0].strip()]+'-'+new_date[1].strip()

In [10]:
fighters_links = json.load(open('sherdog_fighters_link.txt','r'))
fighters_links = ["https://www.sherdog.com"+fighters_links[x] for x in fighters_links]

In [11]:
def get_fighters_info(url):
    fighter = {'Name':'',
               'Nickname':'',
               'Birth date':'',
               'Height':'',
               'Record':{'wins':'','loses':'','draws':'','nc':''},
               'Affiliation':'',
               'Nationality':'',
               'Location region':'',
               'Location city':'',
               'Weight':'',
               'wins summary': {'KO/TKO':'','SUBMISSIONS':'','DECISIONS':'','OTHERS':''},
               'loses summary': {'KO/TKO':'','SUBMISSIONS':'','DECISIONS':'','OTHERS':''},
               'bouts':[]}
    
    driver.get(url)
    site = BeautifulSoup(driver.page_source,'html.parser')
    
    #fighters summary
    bio = site.find('div',class_='module bio_fighter vcard')
    
    
    if not site.find('div',class_='module bio_fighter tagged'):
        if len(bio.find('h1').find_all('span')) > 1:
            fighter['Name'],fighter['Nickname'] = [x.text for x in bio.find('h1').find_all('span')]
        else:
            fighter['Name'] = str(bio.find('h1').find('span').text)
    else:
        fighter['Name'] = str(site.find('span',class_='fn').text)
        fighter['Nickname'] = str(site.find('span',class_='nickname').text if site.find('span',class_='nickname') else '')
        
    
    fighter['Birth date'] = str(bio.find('span',{'itemprop':'birthDate'}).text if bio.find('span',{'itemprop':'birthDate'}) else '')
    if re.search('(\d+\.\d+) cm',bio.find('div',class_='size_info').find('span',class_='item height').text):
        fighter['Height'] = str(re.search('(\d+\.\d+) cm',bio.find('div',class_='size_info').find('span',class_='item height').text)[1])
    if re.search('(\d+\.\d+) kg',bio.find('div',class_='size_info').find('span',class_='item weight').text):
        fighter['Weight'] = str(re.search('(\d+\.\d+) kg',bio.find('div',class_='size_info').find('span',class_='item weight').text)[1])
    fighter['Record']['wins'] = str(bio.find('span',class_='counter').text)
    fighter['Record']['loses'] = str(bio.find('div',class_='bio_graph loser').find('span',class_='counter').text)
    
    if bio.find('div',class_='right_side'):
        for outs in bio.find('div',class_='right_side').find_all('div',class_='bio_graph'):
            if outs.find('span','result').text == 'Draws': fighter['Record']['draws'] = str(outs.find('span','counter').text)
            if outs.find('span','result').text == 'N/C': fighter['Record']['nc'] = str(outs.find('span','counter').text)
    
    bouts_summary = [re.match('\d+',x.text)[0] for x in bio.find_all('span',class_='graph_tag')]
    if len(bouts_summary) == 8:
        fighter['wins summary']['KO/TKO'], fighter['wins summary']['SUBMISSIONS'], fighter['wins summary']['DECISIONS'], fighter['wins summary']['OTHERS'] = bouts_summary[0:4]
        fighter['loses summary']['KO/TKO'], fighter['loses summary']['SUBMISSIONS'], fighter['loses summary']['DECISIONS'], fighter['loses summary']['OTHERS'] = bouts_summary[4:]
    else:
        fighter['wins summary']['KO/TKO'], fighter['wins summary']['SUBMISSIONS'], fighter['wins summary']['DECISIONS'] = bouts_summary[0:3]
        fighter['loses summary']['KO/TKO'], fighter['loses summary']['SUBMISSIONS'], fighter['loses summary']['DECISIONS'] = bouts_summary[3:]
        
    fighter['Nationality'] = str(bio.find('strong',{'itemprop':'nationality'}).text if bio.find('strong',{'itemprop':'nationality'}) else '')
    fighter['Affiliation'] =str(bio.find('a',class_='association').text if bio.find('a',class_='association') else '')
    
    if bio.find('span',class_='locality'):
        if ',' in bio.find('span',class_='locality').text:
            fighter['Location city'], fighter['Location region'] = bio.find('span',class_='locality').text.split(',')
        else:
            fighter['Location city'] = str(bio.find('span',class_='locality').text)
    
    #fights description
    for history in site.find_all('div',class_='module fight_history'):
        if (re.search('Pro Exhibition',history.find('h2').text,re.IGNORECASE) or 
        re.search('Amateur',history.find('h2').text,re.IGNORECASE) or
        re.search('Upcoming',history.find('h2').text,re.IGNORECASE)): continue

        for bout in history.find_all('tr')[1:][::-1]:
            bout_info = {'result':'','opponent':'','event':'','date':'','method':'','referee':'','round':'','time':'','dc reason':""}
            fight_info = bout.find_all('td')
            bout_info['result'] = str(fight_info[0].text)
            bout_info['opponent'] = str(fight_info[1].text)
            bout_info['event'] = str(fight_info[2].a.text)
            bout_info['date'] = str(sherdog_date_decoder(fight_info[2].find('span','sub_line').text) if fight_info[2].find('span','sub_line') else '')
            bout_info['method'] = str(fight_info[3].contents[0])
            if len(fight_info[3].contents) == 2:
                pass
            elif len(fight_info[3].contents[2]) == 1: bout_info['referee'] = str(fight_info[3].contents[2].text if fight_info[3].contents[2] else '')
            else:
                bout_info['referee'] = str(fight_info[3].contents[4].text if str(fight_info[3].contents[4]) != 'N/A' else '')
                bout_info['dc reason'] = str(unidecode(fight_info[3].contents[2]))

            bout_info['round'] = str(fight_info[4].text)
            bout_info['time'] = str(fight_info[5].text)
            fighter['bouts'].append(bout_info)
        
    return fighter

In [12]:
fighters = []

In [13]:
def get_information_about_all_fighters(sherdog_links: list):
    for offset,link in enumerate(sherdog_links):
        fighters.append(get_fighters_info(link))
        print(offset,'\t',link,end=' ')

In [14]:
get_information_about_all_fighters(fighters_links)

0 	 https://www.sherdog.com/fighter/Royce-Gracie-19 1 	 https://www.sherdog.com/fighter/Gerard-Gordeau-15 2 	 https://www.sherdog.com/fighter/Jason-DeLucia-22 3 	 https://www.sherdog.com/fighter/Trent-Jenkins-23 4 	 https://www.sherdog.com/fighter/Ken-Shamrock-4 5 	 https://www.sherdog.com/fighter/Kevin-Rosier-17 6 	 https://www.sherdog.com/fighter/Patrick-Smith-21 7 	 https://www.sherdog.com/fighter/Art-Jimmerson-20 8 	 https://www.sherdog.com/fighter/Zane-Frazier-18 9 	 https://www.sherdog.com/fighter/Teila-Tuli-16 10 	 https://www.sherdog.com/fighter/Remco-Pardoel-33 11 	 https://www.sherdog.com/fighter/Johnny-Rhodes-27 12 	 https://www.sherdog.com/fighter/Orlando-Wiet-31 13 	 https://www.sherdog.com/fighter/Fred-Ettish-36 14 	 https://www.sherdog.com/fighter/Scott-Morris-24 15 	 https://www.sherdog.com/fighter/Minoki-Ichihara-35 16 	 https://www.sherdog.com/fighter/Scott-Baker-34 17 	 https://www.sherdog.com/fighter/Alberto-Cerra-Leon-37 18 	 https://www.sherdog.com/fighter/Robert-

151 	 https://www.sherdog.com/fighter/Joe-Pardo-199 152 	 https://www.sherdog.com/fighter/Bob-Gilstrap-197 153 	 https://www.sherdog.com/fighter/Allan-Goes-196 154 	 https://www.sherdog.com/fighter/Andre-Roberts-194 155 	 https://www.sherdog.com/fighter/Chuck-Liddell-192 156 	 https://www.sherdog.com/fighter/Noe-Hernandez-193 157 	 https://www.sherdog.com/fighter/John-Lober-210 158 	 https://www.sherdog.com/fighter/Wanderlei-Silva-209 159 	 https://www.sherdog.com/fighter/Pedro-Rizzo-208 160 	 https://www.sherdog.com/fighter/Ebenezer-Fontes-Braga-207 161 	 https://www.sherdog.com/fighter/Tulio-Palhares-205 162 	 https://www.sherdog.com/fighter/Adriano-Santos-206 163 	 https://www.sherdog.com/fighter/Cesar-Marcussi-203 164 	 https://www.sherdog.com/fighter/Paulo-Santos-204 165 	 https://www.sherdog.com/fighter/Bas-Rutten-214 166 	 https://www.sherdog.com/fighter/Jorge-Patino-1015 167 	 https://www.sherdog.com/fighter/Evan-Tanner-212 168 	 https://www.sherdog.com/fighter/Darrell-Gholar-2

299 	 https://www.sherdog.com/fighter/Falaniko-Vitale-370 300 	 https://www.sherdog.com/fighter/Jorge-Rivera-2712 301 	 https://www.sherdog.com/fighter/Edwin-Dewees-720 302 	 https://www.sherdog.com/fighter/Karo-Parisyan-5153 303 	 https://www.sherdog.com/fighter/Josh-Thomson-2394 304 	 https://www.sherdog.com/fighter/Gerald-Strebendt-2327 305 	 https://www.sherdog.com/fighter/Nick-Diaz-2831 306 	 https://www.sherdog.com/fighter/Jeremy-Jackson-2651 307 	 https://www.sherdog.com/fighter/Frank-Trigg-311 308 	 https://www.sherdog.com/fighter/Chris-Liguori-4102 309 	 https://www.sherdog.com/fighter/Nick-Agallar-2295 310 	 https://www.sherdog.com/fighter/Renato-Verissimo-4095 311 	 https://www.sherdog.com/fighter/Lee-Murray-1659 312 	 https://www.sherdog.com/fighter/Georges-St-Pierre-3500 313 	 https://www.sherdog.com/fighter/Jeff-Curran-770 314 	 https://www.sherdog.com/fighter/Mike-Kyle-3619 315 	 https://www.sherdog.com/fighter/Jonathan-Wiezorek-2688 316 	 https://www.sherdog.com/fighter

444 	 https://www.sherdog.com/fighter/Gabe-Ruediger-5542 445 	 https://www.sherdog.com/fighter/Joe-Lauzon-4923 446 	 https://www.sherdog.com/fighter/Roger-Huerta-10089 447 	 https://www.sherdog.com/fighter/Jason-Dent-8321 448 	 https://www.sherdog.com/fighter/Eddie-Sanchez-12098 449 	 https://www.sherdog.com/fighter/Mario-Neto-1950 450 	 https://www.sherdog.com/fighter/Tyson-Griffin-11689 451 	 https://www.sherdog.com/fighter/David-Lee-6945 452 	 https://www.sherdog.com/fighter/Chris-Price-14121 453 	 https://www.sherdog.com/fighter/Jason-MacDonald-4389 454 	 https://www.sherdog.com/fighter/Seth-Petruzelli-2738 455 	 https://www.sherdog.com/fighter/Dustin-Hazelett-11481 456 	 https://www.sherdog.com/fighter/Kuniyoshi-Hironaka-2575 457 	 https://www.sherdog.com/fighter/Carmelo-Marrero-12855 458 	 https://www.sherdog.com/fighter/Dan-Lauzon-16556 459 	 https://www.sherdog.com/fighter/Clay-Guida-8184 460 	 https://www.sherdog.com/fighter/Justin-James-1926 461 	 https://www.sherdog.com/figh

588 	 https://www.sherdog.com/fighter/Cale-Yarbrough-34073 589 	 https://www.sherdog.com/fighter/Rory-Markham-10768 590 	 https://www.sherdog.com/fighter/Brodie-Farber-6123 591 	 https://www.sherdog.com/fighter/Nate-Loughran-15626 592 	 https://www.sherdog.com/fighter/Johnny-Rees-14891 593 	 https://www.sherdog.com/fighter/Brad-Blackburn-2920 594 	 https://www.sherdog.com/fighter/James-Giboo-14266 595 	 https://www.sherdog.com/fighter/Shannon-Gugerty-11001 596 	 https://www.sherdog.com/fighter/Dale-Hartt-11970 597 	 https://www.sherdog.com/fighter/Dan-Evensen-10584 598 	 https://www.sherdog.com/fighter/Jon-Jones-27944 599 	 https://www.sherdog.com/fighter/Andre-Gusmao-15806 600 	 https://www.sherdog.com/fighter/Steve-Bruno-4367 601 	 https://www.sherdog.com/fighter/Ryan-Thomas-20224 602 	 https://www.sherdog.com/fighter/Michael-Patt-3506 603 	 https://www.sherdog.com/fighter/Joe-Vedepo-15833 604 	 https://www.sherdog.com/fighter/Jason-Brilz-1927 605 	 https://www.sherdog.com/fighter/Mi

...

1291 	 https://www.sherdog.com/fighter/Guilherme-Vasconcelos-108499 1292 	 https://www.sherdog.com/fighter/Bubba-Bush-29111 1293 	 https://www.sherdog.com/fighter/Corey-Anderson-171723 1294 	 https://www.sherdog.com/fighter/Matt-Van-Buren-64754 1295 	 https://www.sherdog.com/fighter/Eddie-Gordon-66410 1296 	 https://www.sherdog.com/fighter/Dhiego-Lima-59607 1297 	 https://www.sherdog.com/fighter/Carlos-Augusto-Inocente-Filho-17786 1298 	 https://www.sherdog.com/fighter/Juan-Manuel-Puig-44638 1299 	 https://www.sherdog.com/fighter/Patrick-Walsh-67971 1300 	 https://www.sherdog.com/fighter/Dan-Spohn-39485 1301 	 https://www.sherdog.com/fighter/Sarah-Moras-61600 1302 	 https://www.sherdog.com/fighter/Alexis-Dufresne-113687 1303 	 https://www.sherdog.com/fighter/Robert-Drysdale-67894 1304 	 https://www.sherdog.com/fighter/Keith-Berish-46314 1305 	 https://www.sherdog.com/fighter/Jerrod-Sanders-48156 1306 	 https://www.sherdog.com/fighter/Claudia-Gadelha-48404 1307 	 https://www.sherdog.com

1428 	 https://www.sherdog.com/fighter/Garreth-McLellan-71237 1429 	 https://www.sherdog.com/fighter/Mickael-Lebout-81143 1430 	 https://www.sherdog.com/fighter/Damian-Stasiak-86920 1431 	 https://www.sherdog.com/fighter/Aleksandra-Albu-144949 1432 	 https://www.sherdog.com/fighter/Izabela-Badurek-122349 1433 	 https://www.sherdog.com/fighter/Steven-Ray-59838 1434 	 https://www.sherdog.com/fighter/Taylor-Lapilus-97337 1435 	 https://www.sherdog.com/fighter/Rocky-Lee-84854 1436 	 https://www.sherdog.com/fighter/Shane-Campbell-34625 1437 	 https://www.sherdog.com/fighter/Jonavin-Webb-81634 1438 	 https://www.sherdog.com/fighter/Ben-Nguyen-8183 1439 	 https://www.sherdog.com/fighter/Levan-Makashvili-126881 1440 	 https://www.sherdog.com/fighter/Islam-Makhachev-76836 1441 	 https://www.sherdog.com/fighter/Leo-Kuntz-44602 1442 	 https://www.sherdog.com/fighter/Darren-Till-73436 1443 	 https://www.sherdog.com/fighter/Nicolas-Dalby-63608 1444 	 https://www.sherdog.com/fighter/Elizeu-Zaleski-d

1564 	 https://www.sherdog.com/fighter/Eric-Spicely-63723 1565 	 https://www.sherdog.com/fighter/Matthew-Lopez-136139 1566 	 https://www.sherdog.com/fighter/Devin-Clark-72777 1567 	 https://www.sherdog.com/fighter/Bojan-Mihajlovic-12400 1568 	 https://www.sherdog.com/fighter/JC-Cottrell-43874 1569 	 https://www.sherdog.com/fighter/Dmitry-Smoliakov-125781 1570 	 https://www.sherdog.com/fighter/Hector-Sandoval-35409 1571 	 https://www.sherdog.com/fighter/Joe-Gigliotti-160679 1572 	 https://www.sherdog.com/fighter/Danielle-Taylor-123439 1573 	 https://www.sherdog.com/fighter/Jason-Novelli-18136 1574 	 https://www.sherdog.com/fighter/Justin-Ledet-72052 1575 	 https://www.sherdog.com/fighter/Chase-Sherman-159319 1576 	 https://www.sherdog.com/fighter/Mike-Perry-176179 1577 	 https://www.sherdog.com/fighter/Sabah-Homasi-47068 1578 	 https://www.sherdog.com/fighter/Chris-Avila-155641 1579 	 https://www.sherdog.com/fighter/Max-Griffin-53904 1580 	 https://www.sherdog.com/fighter/Marvin-Vettori

1701 	 https://www.sherdog.com/fighter/Azunna-Anyanwu-68352 1702 	 https://www.sherdog.com/fighter/Gokhan-Saki-10925 1703 	 https://www.sherdog.com/fighter/Syuri-Kondo-216929 1704 	 https://www.sherdog.com/fighter/Daichi-Abe-191861 1705 	 https://www.sherdog.com/fighter/Mara-Romero-Borella-164915 1706 	 https://www.sherdog.com/fighter/Kalindra-Faria-48080 1707 	 https://www.sherdog.com/fighter/Poliana-Botelho-164275 1708 	 https://www.sherdog.com/fighter/Jodie-Esquibel-66876 1709 	 https://www.sherdog.com/fighter/Oskar-Piechota-77303 1710 	 https://www.sherdog.com/fighter/Nasrat-Haqparast-122581 1711 	 https://www.sherdog.com/fighter/Ramazan-Emeev-72356 1712 	 https://www.sherdog.com/fighter/Salim-Touahri-73527 1713 	 https://www.sherdog.com/fighter/Aspen-Ladd-141393 1714 	 https://www.sherdog.com/fighter/Marcelo-Golm-190735 1715 	 https://www.sherdog.com/fighter/Karl-Roberson-132605 1716 	 https://www.sherdog.com/fighter/JessicaRose-Clark-115991 1717 	 https://www.sherdog.com/fighter/

1838 	 https://www.sherdog.com/fighter/Bobby-Moffett-159019 1839 	 https://www.sherdog.com/fighter/Devonte-Smith-89163 1840 	 https://www.sherdog.com/fighter/Johnny-Walker-170203 1841 	 https://www.sherdog.com/fighter/Ian-Heinisch-187341 1842 	 https://www.sherdog.com/fighter/Laureano-Staropoli-141271 1843 	 https://www.sherdog.com/fighter/Jesus-Pinedo-146941 1844 	 https://www.sherdog.com/fighter/Anderson-dos-Santos-95053 1845 	 https://www.sherdog.com/fighter/Sergei-Pavlovich-184051 1846 	 https://www.sherdog.com/fighter/Vince-Morales-177019 1847 	 https://www.sherdog.com/fighter/Martin-Day-155883 1848 	 https://www.sherdog.com/fighter/Su-Mudaerji-228263 1849 	 https://www.sherdog.com/fighter/Juan-Francisco-Espino-Dieppa-249511 1850 	 https://www.sherdog.com/fighter/Justin-Frazier-77208 1851 	 https://www.sherdog.com/fighter/Macy-Chiasson-190279 1852 	 https://www.sherdog.com/fighter/Pannie-Kianzad-99523 1853 	 https://www.sherdog.com/fighter/Edmen-Shahbazyan-247691 1854 	 https://ww

In [18]:
len(fighters)

1923

In [20]:
fighters[1750]

{'Name': 'Michal Oleksiejczuk',
 'Nickname': '"Lord"',
 'Birth date': '1995-02-22',
 'Height': '182.88',
 'Record': {'wins': '14', 'loses': '2', 'draws': '', 'nc': '1'},
 'Affiliation': 'Gornik Leczna',
 'Nationality': 'Poland',
 'Location region': '',
 'Location city': 'Barki',
 'Weight': '92.99',
 'wins summary': {'KO/TKO': '10',
  'SUBMISSIONS': '1',
  'DECISIONS': '3',
  'OTHERS': ''},
 'loses summary': {'KO/TKO': '1',
  'SUBMISSIONS': '1',
  'DECISIONS': '0',
  'OTHERS': ''},
 'bouts': [{'result': 'win',
   'opponent': 'Rajmund Flejmer',
   'event': 'Vale Tudo Promotions - Vale Tudo Cup 1',
   'date': '2014-01-11',
   'method': 'Decision (Unanimous)',
   'referee': 'Thomas Jeruszka',
   'round': '2',
   'time': '5:00',
   'dc reason': ''},
  {'result': 'win',
   'opponent': 'Mateusz Gola',
   'event': 'TFN - Tymex Boxing Night',
   'date': '2014-02-08',
   'method': 'Decision (Unanimous)',
   'referee': 'Bogan Pawlik',
   'round': '3',
   'time': '5:00',
   'dc reason': ''},
  {'r

In [178]:
for ind in range(len(cropped)):
    fighter = cropped[ind]
    for stat in fighter:
        if type(fighter[stat]) == dict:
            for level2 in fighter[stat]:
                fighter[stat][level2] = str(fighter[stat][level2])
        if type(fighter[stat]) == list:
            bouts = fighter[stat]
            for bout_num in range(len(bouts)):
                for bout_stat in bouts[bout_num]:
                    bouts[bout_num][bout_stat] = str(bouts[bout_num][bout_stat])
        if type(fighter[stat]) == str:
            fighter[stat] = str(fighter[stat])

In [21]:
json.dump(fighters,open('fighters_sherdog','w'))